In [1]:
import pandas as pd
import numpy as np

from scipy import stats

In [2]:
data = pd.read_csv("churn_analysis.csv")
data.head()

# Измерены следующие признаки:
# state — штат США
# account_length — длительность использования аккаунта
# area_code — деление пользователей на псевдорегионы, использующееся в телекоме
# intl_plan — подключена ли у пользователя услуга международного общения
# vmail_plan — подключена ли у пользователя услуга голосовых сообщений
# vmail_message — количество голосых сообщений, который пользователь отправил / принял
# day_calls — сколько пользователь совершил дневных звонков
# day_mins — сколько пользователь проговорил минут в течение дня
# day_charge — сколько пользователь заплатил за свою дневную активность
# eve_calls, eve_mins, eve_charge — аналогичные метрики относительно вечерней активности
# night_calls, night_mins, night_charge — аналогичные метрики относительно ночной активности
# intl_calls, intl_mins, intl_charge — аналогичные метрики относительно международного общения
# custserv_calls — сколько раз пользователь позвонил в службу поддержки
# treatment — номер стратегии, которая применялись для удержания абонентов (0, 2 = два разных типа воздействия, 1 = контрольная группа)
# mes_estim — оценка интенсивности пользования интернет мессенджерами
# churn — результат оттока: перестал ли абонент пользоваться услугами оператора

,Unnamed: 0,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,0,KS,128,415,no,yes,25,265.1,110,45.07,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,1,OH,107,415,no,yes,26,161.6,123,27.47,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,2,NJ,137,415,no,no,0,243.4,114,41.38,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,3,OH,84,408,yes,no,0,299.4,71,50.90,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,4,OK,75,415,yes,no,0,166.7,113,28.34,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.


In [7]:
# 1. Давайте рассмотрим всех пользователей из контрольной группы (treatment = 1). 
# Для таких пользователей мы хотим проверить гипотезу о том, что штат абонента не влияет на то, 
# перестанет ли абонент пользоваться услугами оператора. Для этого мы воспользуемся критерием хи-квадрат. 
# Постройте таблицы сопряженности между каждой из всех 1275 возможных неупорядоченных пар штатов и значением признака churn.
# Сколько достигаемых уровней значимости оказались меньше, чем α=0.05?


In [2]:
pt = data.loc[data.treatment == 1, :].pivot_table(index="state", 
     values="area_code", columns="churn", aggfunc = len, fill_value=0

In [11]:
states = list(set(data['state'].values))
# states

In [12]:
pairs = [(states[i], states[j]) for i in range(len(states)-1) for j in range(i+1,len(states))]
# pairs

In [18]:
count1 = 0
count2 = 0
alpha = 0.05
pvals_chi1=[]
pvals_chi2=[]
pvals_fisher=[]

for s1, s2 in pairs:
    pval1 = stats.chi2_contingency(pt.loc[[s1, s2], :], correction=False)[1]
    pvals_chi1.append(pval1)
    if pval1<alpha:
        count1+=1
        
    pval2 = stats.chi2_contingency(pt.loc[[s1, s2], :], correction=True)[1]
    pvals_chi2.append(pval2)
    if pval2<alpha:
        count2+=1
        
    pval3 = stats.fisher_exact(pt.loc[[s1, s2], :])[1]
    pvals_fisher.append(pval3)
        
print (count1)
print (count2)
print (np.mean(pvals_chi1))
print (np.mean(pvals_chi2))
print (np.mean(pvals_fisher))

34
0
0.5018273798739158
0.6640566382051046
0.6483383060020681


In [19]:
# 5. Давайте попробуем применить полученные знания о разных видах корреляции и ее применимости на практике.
# Рассмотрим пару признаков day_calls и mes_estim. 
# Посчитайте корреляцию Пирсона между этими признаками на всех данных, ее значимость.

stats.pearsonr(data['day_calls'], data['mes_estim'])

(-0.051794350587572625, 0.0027798836869756707)

In [20]:
# 6. Еще раз рассмотрим пару признаков day_calls и mes_estim. 
#  Посчитайте корреляцию Спирмена между этими признаками на всех данных, ее значимость.

stats.spearmanr(data['day_calls'], data['mes_estim'])

SpearmanrResult(correlation=0.043349880533927444, pvalue=0.012317367189170541)

In [24]:
# 8. Посчитайте значение коэффицента корреляции Крамера между двумя признаками: штатом (state) и оттоком пользователей (churn) 
# для всех пользователей, которые находились в контрольной группе (treatment=1). 
# Что можно сказать о достигаемом уровне значимости при проверке гипотезы о равенство нулю этого коэффициента?

# chi2 = stats.chi2_contingency(happy)[0]
n1 = np.sum(pvals_chi1)
n2 = np.sum(pvals_chi2)
V_Kr1 = np.sqrt(pvals_chi1/(n1*2)) 
V_Kr2 = np.sqrt(pvals_chi2/(n2*2)) 
print (np.mean(V_Kr1))
print (np.mean(V_Kr2))

0.018830492220466687
0.019319639369369833
